In [1]:
import csv
from bpemb import BPEmb
import torch
import torch.autograd as autograd
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
from torch.nn.init import xavier_uniform_
import copy
from typing import Optional, Any

In [2]:
torch.cuda.empty_cache()

In [3]:
import re
bpemb_sa = BPEmb(lang="sa", vs=100000,dim=300,add_pad_emb=True)
reg = re.compile("[!@#$%^&*,/\_-`~\n]")
sansList = []
with open('allData.csv', encoding = "utf-8") as csvfile:
    spamreader = csv.reader(csvfile)
    for i in spamreader:
        if i != "SANS":
            sansList.append(bpemb_sa.encode(reg.sub(" ",i[1][7:][:-4])))

In [4]:
len(sansList)

18300

In [5]:
# bpemb_sa = BPEmb(lang="sa", vs=100000,dim=300,add_pad_emb=True)
# encodedSansList = []
# for i in sansList:
#     encodedSansList.append(autograd.Variable(torch.tensor([[bpemb_sa.encode_ids(i)]])))

In [6]:
rege = re.compile("[!@#$%^&*:,/\_`~\n]")
engList = []
bpemb_en = BPEmb(lang="en", vs=100000,dim=300, add_pad_emb= True)
with open('allData.csv', encoding = "utf-8") as csvfile:
    spamreader = csv.reader(csvfile)
    for i in spamreader:
        if i != "ENG":
            engList.append(bpemb_en.encode(rege.sub(" ",i[2][7:][:-4])))

In [7]:
from sklearn.model_selection import train_test_split
sans_train, sans_test, eng_train, eng_test = train_test_split(sansList,engList,test_size=0.20,random_state=123)

In [8]:
len(eng_train)

14640

In [9]:
eng_vocab = {}
for each in eng_train:
    for i in each:
        if i not in eng_vocab.keys():
            eng_vocab[i] = len(eng_vocab) + 4

In [10]:
sans_vocab = {}
for each in sans_train:
    for i in each:
        if i not in sans_vocab.keys():
            sans_vocab[i] = len(sans_vocab) + 2

In [11]:
src_vocab_len, tgt_vocab_len = len(sans_vocab), len(eng_vocab)

In [12]:
sans_vocab["<pad>"] = 0
eng_vocab["<pad>"] = 0

In [13]:
# bpemb_en = BPEmb(lang="en", vs=100000,dim=300, add_pad_emb= True)
# encodedEngList = []
# for i in engList:
#     encodedEngList.append(autograd.Variable(torch.tensor([[bpemb_en.encode_ids(i)]])))

In [14]:
eng_vocab["<bos>"] = 1
eng_vocab["<eos>"] = 2
sans_vocab["<unk>"] = 1
eng_vocab["<unk>"] = 3
src_vocab_len, tgt_vocab_len = len(sans_vocab), len(eng_vocab)
print(src_vocab_len, tgt_vocab_len)

21828 18000


In [82]:
list(eng_vocab.keys())[list(eng_vocab.values()).index(1345)]

'▁observing'

In [15]:
batch_size = 4
batch_size_test = 1

In [16]:
entireSansTensorList = []
for i in range(0, len(sans_train), batch_size):
    temp1 = []
    longest = len(max(sans_train[i:i+batch_size], key=len))
    #print(longest)
    for a in sans_train[i:i+batch_size]:
        temp2 = []
        for e in a:  
            temp2.append(sans_vocab.get(e))
        zeros = [0] * (longest - len(a))
        temp1.append(temp2 + zeros)
    entireSansTensorList.append(autograd.Variable(torch.tensor(temp1)).transpose(0,1).long())

In [17]:
entireSansTestTensorList = []
for i in range(0, len(sans_test), batch_size_test):
    temp1 = []
    longest = len(max(sans_test[i:i+batch_size_test], key=len))
    #print(longest)
    for a in sans_test[i:i+batch_size_test]:
        temp2 = []
        for e in a:  
            temp2.append(sans_vocab.get(e,1))
        zeros = [0] * (longest - len(a))
        temp1.append(temp2 + zeros)
    entireSansTestTensorList.append(autograd.Variable(torch.tensor(temp1)).transpose(0,1).long())

In [18]:
entireSansTensorList[1].size()

torch.Size([26, 4])

In [19]:
entireEngTensorList = []
for i in range(0, len(eng_train), batch_size):
    temp1 = []
    longest = len(max(eng_train[i:i+batch_size], key=len))
#     print(longest)
    for a in eng_train[i:i+batch_size]:
        temp2 = []
        for e in a:  
            temp2.append(eng_vocab.get(e))
        zeros = [0] * (longest - len(a))
        temp1.append([1] + temp2 + [2] + zeros)
    entireEngTensorList.append(autograd.Variable(torch.tensor(temp1)).transpose(0,1).long())

In [20]:
entireEngTestTensorList = []
for i in range(0, len(eng_test), batch_size_test):
    temp1 = []
    longest = len(max(eng_test[i:i+batch_size_test], key=len))
#     print(longest)
    for a in eng_test[i:i+batch_size_test]:
        temp2 = []
        for e in a:  
            temp2.append(eng_vocab.get(e,3))
        zeros = [0] * (longest - len(a))
        temp1.append([1] + temp2 + [2] + zeros)
    entireEngTestTensorList.append(autograd.Variable(torch.tensor(temp1)).transpose(0,1).long())

In [21]:
entireEngTensorList[0].size(1)

4

In [22]:
sans_train_tensor, sans_val_tensor, eng_train_tensor, eng_val_tensor = train_test_split(entireSansTensorList,entireEngTensorList,test_size=0.10,random_state=123)

In [23]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.d_model = d_model
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x * math.sqrt(self.d_model)
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class MyTransformer(nn.Module):
    def __init__(self, d_model: int = 512, nhead: int = 8, num_encoder_layers: int = 6,
                 num_decoder_layers: int = 6, dim_feedforward: int = 2048, dropout: float = 0.1,
                 activation: str = "relu",source_vocab_length: int = 60000,target_vocab_length: int = 60000) -> None:
        super(MyTransformer, self).__init__()
        self.source_embedding = nn.Embedding(source_vocab_length, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, activation)
        encoder_norm = nn.LayerNorm(d_model)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_encoder_layers, encoder_norm)
        self.target_embedding = nn.Embedding(target_vocab_length, d_model)
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout, activation)
        decoder_norm = nn.LayerNorm(d_model)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_decoder_layers, decoder_norm)
        self.out = nn.Linear(512, target_vocab_length)
        self._reset_parameters()
        self.d_model = d_model
        self.nhead = nhead

    def forward(self, src: Tensor, tgt: Tensor, src_mask: Optional[Tensor] = None, tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None, src_key_padding_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None, memory_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        if src.size(1) != tgt.size(1):
            raise RuntimeError("the batch number of src and tgt must be equal")
        src = self.source_embedding(src)
        src = self.pos_encoder(src)
        memory = self.encoder(src, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        tgt = self.target_embedding(tgt)
        tgt = self.pos_encoder(tgt)
        output = self.decoder(tgt, memory, tgt_mask=tgt_mask, memory_mask=memory_mask,
                              tgt_key_padding_mask=tgt_key_padding_mask,
                              memory_key_padding_mask=memory_key_padding_mask)
        output = self.out(output)
        return output


    def _reset_parameters(self):
        r"""Initiate parameters in the transformer model."""
        for p in self.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

In [24]:
model = MyTransformer(source_vocab_length=src_vocab_len,target_vocab_length=src_vocab_len)
model = model.to("cuda")
optim = torch.optim.Adam(model.parameters(), lr=0.00001, betas=(0.9, 0.98), eps=1e-9)
#optim = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [25]:
def train(sans_train, eng_train, sans_test, eng_test, model, optim, num_epochs,use_gpu=True): 
    train_losses = []
    valid_losses = []
    for epoch in range(num_epochs):
        train_loss = 0
        valid_loss = 0
        # Train model
        model.train()
        for i in range(len(sans_train)):
            #m = torch.zeros(1,1,512-(sans_train[i].size(2)))
            #print(i)
            src = sans_train[i]
            #src = torch.cat((sans_train[i],m),2).long()
            #n = torch.zeros(1, 1, 512 - (eng_train[i].size(2))-2)
            trg = eng_train[i]
            #trg = torch.cat((eng_new,n),2).long()
#             print(src.size(), trg.size())
            #change to shape (bs , max_seq_len)
            src = src.transpose(0,1)
            #change to shape (bs , max_seq_len+1) , Since right shifted
            trg = trg.transpose(0,1)
            trg_input = trg[:, :-1]
            targets = trg[:, 1:].contiguous().view(-1)
#             src_mask = (src != 0)
#             src_mask = src_mask.float().masked_fill(src_mask == 0, float('-inf')).masked_fill(src_mask == 1, float(0.0))
#             src_mask = src_mask.cuda() if use_gpu else src_mask
#             trg_mask = (trg_input != 0)
#             trg_mask = trg_mask.float().masked_fill(trg_mask == 0, float('-inf')).masked_fill(trg_mask == 1, float(0.0))
#             trg_mask = trg_mask.cuda() if use_gpu else trg_mask
            size = trg_input.size(1)
            #print(size)
            np_mask = torch.triu(torch.ones(size, size)==1).transpose(0,1)
            np_mask = np_mask.float().masked_fill(np_mask == 0, float('-inf')).masked_fill(np_mask == 1, float(0.0))
            np_mask = np_mask.cuda() if use_gpu else np_mask   
            # Forward, backprop, optimizer
            optim.zero_grad()
            preds = model(src.transpose(0,1).to("cuda"), trg_input.transpose(0,1).to("cuda"))#, src_mask = src_mask)#, tgt_key_padding_mask=trg_mask)
            preds = preds.transpose(0,1).contiguous().view(-1, preds.size(-1))
            loss = F.cross_entropy(preds.to('cuda'),targets.to('cuda'), ignore_index=0,reduction='sum')
            loss.backward()
            optim.step()
            train_loss += loss.item()/src.size(1)
            print("----------- Batch: " + str(i) + " loss: " + str(train_loss))
        model.eval()
        with torch.no_grad():
            for i in range(len(sans_test)):
                src = sans_test[i]
                #src = torch.cat((sans_train[i],m),2).long()
                #n = torch.zeros(1, 1, 512 - (eng_train[i].size(2))-2)
                trg = eng_test[i]
                src = src.transpose(0,1)
                #change to shape (bs , max_seq_len+1) , Since right shifted
                trg = trg.transpose(0,1)
                trg_input = trg[:, :-1]
                targets = trg[:, 1:].contiguous().view(-1)
#                 src_mask = (src != 0)
#                 src_mask = src_mask.float().masked_fill(src_mask == 0, float('-inf')).masked_fill(src_mask == 1, float(0.0))
#                 src_mask = src_mask.cuda() if use_gpu else src_mask
                trg_mask = (trg_input != 0)
                trg_mask = trg_mask.float().masked_fill(trg_mask == 0, float('-inf')).masked_fill(trg_mask == 1, float(0.0))
                trg_mask = trg_mask.cuda() if use_gpu else trg_mask
                size = trg_input.size(1)
                #print(size)
                np_mask = torch.triu(torch.ones(size, size)==1).transpose(0,1)
                np_mask = np_mask.float().masked_fill(np_mask == 0, float('-inf')).masked_fill(np_mask == 1, float(0.0))
                np_mask = np_mask.cuda() if use_gpu else np_mask

                preds = model(src.transpose(0,1).to('cuda'), trg_input.transpose(0,1).to('cuda'), tgt_mask = np_mask)#, src_mask = src_mask)#, tgt_key_padding_mask=trg_mask)
                preds = preds.transpose(0,1).contiguous().view(-1, preds.size(-1))         
                loss = F.cross_entropy(preds.to('cuda'),targets.to('cuda'), ignore_index=0,reduction='sum')
                valid_loss += loss.item()/src.size(1)
            
        # Log after each epoch
        print(f'''Epoch [{epoch+1}/{num_epochs}] complete. Train Loss: {train_loss/src.size(1):.3f}. Val Loss: {valid_loss/src.size(1):.3f}''')
        
        #Save best model till now:
        if valid_loss/len(sans_test)<min(valid_losses,default=1e9): 
            print("saving state dict")
            torch.save(model.state_dict(), f"checkpoint_best_epoch.pt")
        
        train_losses.append(train_loss/src.size(1))
        valid_losses.append(valid_loss/src.size(1))
        
#         # Check Example after each epoch:
# #         sentences = ["This is an example to check how our model is performing."]
# #         for sentence in sentences:
# #             print(f"Original Sentence: {sentence}")
# #             print(f"Translated Sentence: {greeedy_decode_sentence(model,sentence)}")
    return train_losses,valid_losses

In [26]:
train_losses,valid_losses = train(sans_train_tensor, eng_train_tensor, sans_val_tensor ,eng_val_tensor, model, optim, 10)

----------- Batch: 0 loss: 34.53425357216283
----------- Batch: 1 loss: 84.39190275050335
----------- Batch: 2 loss: 120.14573388031104
----------- Batch: 3 loss: 147.88889116112702
----------- Batch: 4 loss: 196.363207567377
----------- Batch: 5 loss: 235.79989503685943
----------- Batch: 6 loss: 291.7972564401046
----------- Batch: 7 loss: 324.5117361986273
----------- Batch: 8 loss: 356.6901118104496
----------- Batch: 9 loss: 396.3126425024139
----------- Batch: 10 loss: 438.32113108414467
----------- Batch: 11 loss: 467.5430919969216
----------- Batch: 12 loss: 513.6680835782794
----------- Batch: 13 loss: 560.3888346594736
----------- Batch: 14 loss: 609.5436991614267
----------- Batch: 15 loss: 659.8389375150147
----------- Batch: 16 loss: 691.1445235325608
----------- Batch: 17 loss: 720.9741640470065
----------- Batch: 18 loss: 758.3805284879245
----------- Batch: 19 loss: 807.8088234821374
----------- Batch: 20 loss: 846.6858437458093
----------- Batch: 21 loss: 884.445927262

----------- Batch: 176 loss: 7335.3604955618075
----------- Batch: 177 loss: 7365.46408054494
----------- Batch: 178 loss: 7409.264030777813
----------- Batch: 179 loss: 7437.445002524389
----------- Batch: 180 loss: 7455.375588461889
----------- Batch: 181 loss: 7507.771003669324
----------- Batch: 182 loss: 7549.040248523591
----------- Batch: 183 loss: 7587.134582717229
----------- Batch: 184 loss: 7626.132128683015
----------- Batch: 185 loss: 7654.684935780755
----------- Batch: 186 loss: 7682.3435051166925
----------- Batch: 187 loss: 7719.05299655299
----------- Batch: 188 loss: 7773.579755382743
----------- Batch: 189 loss: 7813.478061770185
----------- Batch: 190 loss: 7869.491027381235
----------- Batch: 191 loss: 7923.7550279070765
----------- Batch: 192 loss: 7964.4468522478965
----------- Batch: 193 loss: 8005.568818574575
----------- Batch: 194 loss: 8047.0371250524395
----------- Batch: 195 loss: 8083.783007953718
----------- Batch: 196 loss: 8124.381927990483
----------

----------- Batch: 349 loss: 13780.630991770218
----------- Batch: 350 loss: 13805.71487696309
----------- Batch: 351 loss: 13844.328144128054
----------- Batch: 352 loss: 13885.393867688528
----------- Batch: 353 loss: 13912.951590450353
----------- Batch: 354 loss: 13932.20143707996
----------- Batch: 355 loss: 13973.109631485653
----------- Batch: 356 loss: 14013.36311680354
----------- Batch: 357 loss: 14049.698991779722
----------- Batch: 358 loss: 14089.544395277591
----------- Batch: 359 loss: 14126.044495549633
----------- Batch: 360 loss: 14156.083796377387
----------- Batch: 361 loss: 14179.216983506967
----------- Batch: 362 loss: 14214.150447788454
----------- Batch: 363 loss: 14259.578168600086
----------- Batch: 364 loss: 14283.303964091623
----------- Batch: 365 loss: 14322.236047786646
----------- Batch: 366 loss: 14359.461286586695
----------- Batch: 367 loss: 14404.666983201278
----------- Batch: 368 loss: 14433.455530717147
----------- Batch: 369 loss: 14461.82558545

----------- Batch: 522 loss: 19641.32801921358
----------- Batch: 523 loss: 19671.41841946059
----------- Batch: 524 loss: 19691.477175335225
----------- Batch: 525 loss: 19715.507600139914
----------- Batch: 526 loss: 19734.696135557744
----------- Batch: 527 loss: 19772.00321563587
----------- Batch: 528 loss: 19807.40235799348
----------- Batch: 529 loss: 19830.419191787307
----------- Batch: 530 loss: 19864.019024175377
----------- Batch: 531 loss: 19916.27051264564
----------- Batch: 532 loss: 19955.511069635035
----------- Batch: 533 loss: 19994.403277611345
----------- Batch: 534 loss: 20027.48595499214
----------- Batch: 535 loss: 20065.39493936714
----------- Batch: 536 loss: 20102.109914229695
----------- Batch: 537 loss: 20125.632343196397
----------- Batch: 538 loss: 20157.889294023604
----------- Batch: 539 loss: 20176.16933831157
----------- Batch: 540 loss: 20203.439229512493
----------- Batch: 541 loss: 20243.01433221993
----------- Batch: 542 loss: 20258.527252029555
-

----------- Batch: 694 loss: 25227.678903186676
----------- Batch: 695 loss: 25260.99419437132
----------- Batch: 696 loss: 25295.558162605914
----------- Batch: 697 loss: 25324.90261135322
----------- Batch: 698 loss: 25352.465371769886
----------- Batch: 699 loss: 25388.440374821643
----------- Batch: 700 loss: 25421.371305404326
----------- Batch: 701 loss: 25448.542717379
----------- Batch: 702 loss: 25477.914528292087
----------- Batch: 703 loss: 25505.643841188816
----------- Batch: 704 loss: 25528.211504763036
----------- Batch: 705 loss: 25561.436438496294
----------- Batch: 706 loss: 25574.624539256616
----------- Batch: 707 loss: 25607.955358222134
----------- Batch: 708 loss: 25638.741213902034
----------- Batch: 709 loss: 25676.492648228206
----------- Batch: 710 loss: 25712.250081889306
----------- Batch: 711 loss: 25737.70020580219
----------- Batch: 712 loss: 25770.368583652697
----------- Batch: 713 loss: 25797.00507182544
----------- Batch: 714 loss: 25827.21984515026


...

----------- Batch: 2493 loss: 16276.242330448655
----------- Batch: 2494 loss: 16282.93486532288
----------- Batch: 2495 loss: 16287.566444172904
----------- Batch: 2496 loss: 16291.955195565537
----------- Batch: 2497 loss: 16296.901694041762
----------- Batch: 2498 loss: 16301.114225598236
----------- Batch: 2499 loss: 16306.138908560906
----------- Batch: 2500 loss: 16311.948446008322
----------- Batch: 2501 loss: 16318.559159825778
----------- Batch: 2502 loss: 16328.457426209357
----------- Batch: 2503 loss: 16336.023669874328
----------- Batch: 2504 loss: 16342.765307160345
----------- Batch: 2505 loss: 16349.439650362518
----------- Batch: 2506 loss: 16354.118853129823
----------- Batch: 2507 loss: 16363.391828400165
----------- Batch: 2508 loss: 16372.75034300135
----------- Batch: 2509 loss: 16382.597790889953
----------- Batch: 2510 loss: 16390.872326111792
----------- Batch: 2511 loss: 16395.195402713172
----------- Batch: 2512 loss: 16406.345596881263
----------- Batch: 251

----------- Batch: 2662 loss: 17340.748383422288
----------- Batch: 2663 loss: 17346.408648314864
----------- Batch: 2664 loss: 17353.16486419066
----------- Batch: 2665 loss: 17358.100357151274
----------- Batch: 2666 loss: 17363.385646568888
----------- Batch: 2667 loss: 17366.75673175255
----------- Batch: 2668 loss: 17372.971251477677
----------- Batch: 2669 loss: 17377.35895758298
----------- Batch: 2670 loss: 17383.494940944118
----------- Batch: 2671 loss: 17392.376849106553
----------- Batch: 2672 loss: 17398.353028963073
----------- Batch: 2673 loss: 17403.35642658677
----------- Batch: 2674 loss: 17409.908457451285
----------- Batch: 2675 loss: 17416.420265406512
----------- Batch: 2676 loss: 17423.530481744572
----------- Batch: 2677 loss: 17429.979638369503
----------- Batch: 2678 loss: 17433.15176739201
----------- Batch: 2679 loss: 17437.96813055837
----------- Batch: 2680 loss: 17442.58366675966
----------- Batch: 2681 loss: 17450.31655429081
----------- Batch: 2682 loss

----------- Batch: 2831 loss: 18371.754782005548
----------- Batch: 2832 loss: 18376.386742892377
----------- Batch: 2833 loss: 18382.351416483758
----------- Batch: 2834 loss: 18387.87445463945
----------- Batch: 2835 loss: 18397.758962560456
----------- Batch: 2836 loss: 18404.91242355155
----------- Batch: 2837 loss: 18408.112958219517
----------- Batch: 2838 loss: 18417.25973646749
----------- Batch: 2839 loss: 18421.280211855064
----------- Batch: 2840 loss: 18428.568099428307
----------- Batch: 2841 loss: 18443.68199102987
----------- Batch: 2842 loss: 18451.057034263104
----------- Batch: 2843 loss: 18459.70306468913
----------- Batch: 2844 loss: 18466.45025537652
----------- Batch: 2845 loss: 18471.641293068074
----------- Batch: 2846 loss: 18477.975617408894
----------- Batch: 2847 loss: 18484.223175590243
----------- Batch: 2848 loss: 18492.42742211124
----------- Batch: 2849 loss: 18499.52918424706
----------- Batch: 2850 loss: 18505.989714555377
----------- Batch: 2851 loss

----------- Batch: 3000 loss: 19458.397852189897
----------- Batch: 3001 loss: 19463.782650875608
----------- Batch: 3002 loss: 19474.412477471516
----------- Batch: 3003 loss: 19481.497874697354
----------- Batch: 3004 loss: 19487.189838175724
----------- Batch: 3005 loss: 19493.64274544454
----------- Batch: 3006 loss: 19501.710577301405
----------- Batch: 3007 loss: 19511.229788681976
----------- Batch: 3008 loss: 19515.72534688899
----------- Batch: 3009 loss: 19522.553651038477
----------- Batch: 3010 loss: 19534.992516113223
----------- Batch: 3011 loss: 19541.838916676534
----------- Batch: 3012 loss: 19546.301576315036
----------- Batch: 3013 loss: 19552.45965454402
----------- Batch: 3014 loss: 19555.525649178882
----------- Batch: 3015 loss: 19562.228896249195
----------- Batch: 3016 loss: 19569.32992957439
----------- Batch: 3017 loss: 19576.358865324713
----------- Batch: 3018 loss: 19580.917230510768
----------- Batch: 3019 loss: 19589.460989326955
----------- Batch: 3020 

----------- Batch: 3169 loss: 20535.4731592391
----------- Batch: 3170 loss: 20545.110597332012
----------- Batch: 3171 loss: 20549.191148479473
----------- Batch: 3172 loss: 20553.275600675075
----------- Batch: 3173 loss: 20561.474972557924
----------- Batch: 3174 loss: 20569.55708864679
----------- Batch: 3175 loss: 20575.35012442647
----------- Batch: 3176 loss: 20582.44277474133
----------- Batch: 3177 loss: 20588.770714840706
----------- Batch: 3178 loss: 20594.19159335701
----------- Batch: 3179 loss: 20600.112482639237
----------- Batch: 3180 loss: 20605.966683078692
----------- Batch: 3181 loss: 20611.29718641528
----------- Batch: 3182 loss: 20616.380843574025
----------- Batch: 3183 loss: 20623.44015222655
----------- Batch: 3184 loss: 20630.20445988443
----------- Batch: 3185 loss: 20634.259469787878
----------- Batch: 3186 loss: 20639.089414105034
----------- Batch: 3187 loss: 20644.389506212632
----------- Batch: 3188 loss: 20651.13349424974
----------- Batch: 3189 loss: 

In [27]:
torch.save(model.state_dict(), f"after_10_epochs.pt")

In [91]:
import numpy as np

def decoder(pred):
    vals = []
    for i in range(len(pred)):
        each = pred[i]
        idxs = np.argmax(each, axis=1)
        vals.append(list(eng_vocab.keys())[list(eng_vocab.values()).index(idxs[3])])
    print(vals)
    return bpemb_en.decode(vals)
    

In [84]:
model.eval()
out = model(sans_val_tensor[0].to('cuda'), eng_val_tensor[0].to('cuda'))

In [85]:
print(out.size())
print(len(eng_val_tensor[0]))

torch.Size([40, 4, 21828])
40


In [86]:
print(decoder(out.to('cpu').detach().numpy()))

--- 1345
--- 1431
--- 3731
--- 19
--- 419
--- 19
--- 2894
--- 741
--- 580
--- 117
--- 392
--- 16
--- 24
--- 2620
--- 392
--- 9
--- 2662
--- 19
--- 9
--- 672
--- 29
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
--- 2
['▁observing', '▁various', '▁types', '▁of', '▁thousands', '▁of', '▁reaching', '▁rocks', '▁bharata', '▁have', '▁along', '▁with', '▁his', '▁troops', '▁along', '▁the', '▁side', '▁of', '▁the', '▁mountain', '.', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>']
observing various types of thousands of reaching rocks bharata have along with his troops along the side of the mountain.<eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>


In [88]:
print(decoder(out.to('cpu').detach().numpy()))

['▁hat', '▁great', '▁han', 'uma', '▁saw', '▁the', '▁grew', '▁king', '▁of', '▁ra', 'has', 'has', 'as', '▁in', '▁sleep', '▁on', '▁a', '▁shining', '▁mace', '▁grew', '▁after', '▁whom', '.', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>']
hat great hanuma saw the grew king of rahashasas in sleep on a shining mace grew after whom.<eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos><eos>


In [90]:
print(decoder(out.to('cpu').detach().numpy()))

['▁"', 'va', 'ide', 'hi', '▁who', '▁is', '▁dear', 'er', '▁to', '▁me', '▁than', '▁my', '▁lives', '▁will', '▁she', '▁be', '▁yourself', '▁must', '▁oh', '▁valiant', '▁lakshmana', '▁this', '▁grew', 'whether', '▁of', '▁mine', '▁will', '▁it', '▁not', '▁become', '▁del', '▁some', '▁entirety', '▁', '[', '0-00', '-0', ']', '<eos>', '<eos>']
"vaidehi who is dearer to me than my lives will she be yourself must oh valiant lakshmana this grewwhether of mine will it not become del some entirety [0-00-0]<eos><eos>


In [92]:
print(decoder(out.to('cpu').detach().numpy()))

['▁"', 'that', '▁bharata', '▁the', '▁virtuous', '▁man', '▁who', '▁was', '▁intended', '▁to', '▁doubt', '▁a', '▁glorious', 'ala', '▁the', '▁great', '-', 'armed', '▁and', '▁the', '▁one', 'eight', '▁with', '▁the', '▁truth', '▁is', '▁why', '▁on', '▁my', '▁account', '."', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>', '<eos>']
"that bharata the virtuous man who was intended to doubt a gloriousala the great-armed and the oneeight with the truth is why on my account."<eos><eos><eos><eos><eos><eos><eos><eos><eos>
